# Selection of Hyperparameters

Import relevant modules

In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import pywt
import tensorflow as tf
from tensorflow import keras
import kerastuner as kt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [ ]:
keras.backend.clear_session()

Read coefficient

In [ ]:
data_all = pd.read_csv(r'C:\Users\AliMS\Documents\project\code\data\10_2018_dwt_a3.csv', usecols = ['a3'])

Normalise data

In [ ]:
scaler = MinMaxScaler()
data_all = scaler.fit_transform(data_all)

Select data proportions. Select how far ahead to predict using 'steps_ahead'

In [ ]:
train_size = int(data_all.shape[0]*0.9)
data_train = data_all[:train_size, :]
data_test = data_all[train_size:, :]
steps_ahead = 1

Assign train, validate and test data

In [ ]:
X_train, Y_train = data_train[: - steps_ahead, :], data_train[steps_ahead:, :]
X_test, Y_test = data_test[: - steps_ahead, :], data_test[steps_ahead:, :]
X_all, Y_all = data_all[: - steps_ahead, :], data_all[steps_ahead:, :]

Reshape arrays

In [ ]:
X_train = X_train.reshape(-1, 1, 1)
X_test = X_test.reshape(-1, 1, 1)
Y_train = Y_train.reshape(-1, 1, 1)
X_all = X_all.reshape(-1, 1, 1)

Define function to build model

In [ ]:
def build_model(hp):
    model = keras.Sequential()
    model.add(keras.layers.LSTM(20, return_sequences = True, input_shape = [None, 1]))
    model.add(keras.layers.LSTM(units=hp.Int('units',
                                        min_value=32,
                                        max_value=512,
                                        step=32),
                           activation='sigmoid'))
    model.add(keras.layers.Dense(1, activation = 'sigmoid'))
    hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4]) 
    model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate), loss = 'mse', metrics = ['accuracy'])
    return model

Define Tuner

In [ ]:
tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs = 20,
    overwrite = True)

Run tuner

In [ ]:
tuner.search(X_train, Y_train, validation_data = (X_test, Y_test), epochs=20, callbacks=[tf.keras.callbacks.EarlyStopping(patience=1)])

Select hyperparamters

In [ ]:
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

Print learning rate

In [ ]:
best_hps.get('learning_rate')

Print number of units in hidden layer

In [ ]:
best_hps.get('units')